## 8. Make sure you have copied the workspace archive (.zip) from the aggregator node to the collaborator node.

In [ ]:
!pip install paramiko scp

from paramiko import SSHClient, AutoAddPolicy
from scp import SCPClient
from pathlib import Path
from getpass import getpass
import os

afqdn =  # 'example.domain.com'
workspace_src = # 'path/to/workspace'
archive_name = 'workspace.zip'
folder_name = archive_name.split('.')[0]
workspace_dst = # 'root path of workspace on destination machine'
src_user = 'user'


ssh_client = SSHClient()
ssh_client.set_missing_host_key_policy(AutoAddPolicy())
ssh_client.connect(afqdn, username=src_user, password=getpass(f'Enter {src_user}@{afqdn} password:'))
scp = SCPClient(ssh_client.get_transport())
os.makedirs(workspace_dst, exist_ok=True)
scp.get(str(Path(workspace_src) / archive_name), workspace_dst)

# 10. Import the workspace archive.

In [ ]:
%cd $workspace_dst
!rm -rf $folder_name
!fx workspace import --archive $archive_name
%cd $folder_name

## 12. Create a collaborator certificate request.

In [ ]:
!fx collaborator generate-cert-request -n one -d 1
scp.put('col_one_to_agg_cert_request.zip', workspace_src)

## 15. Get signed certificates from the aggregator.

In [ ]:
scp.get(str(Path(workspace_src) / 'agg_to_col_one_signed_cert.zip'), '.')
!fx collaborator certify --import agg_to_col_one_signed_cert.zip

# Choose a device

In [ ]:
%env CUDA_VISIBLE_DEVICES=0

# Run the collaborator process

In [ ]:
!fx collaborator start -n one